# LCEL (LangChain Expression Language) 메모리 통합

이 노트북에서는 LCEL 체인에 메모리를 추가하여 대화 내용을 기억하는 방법을 학습합니다.

## 주요 학습 내용
- ConversationBufferMemory를 LCEL 체인에 통합하는 방법
- RunnablePassthrough와 RunnableLambda를 활용한 메모리 연결
- MessagesPlaceholder를 사용한 대화 기록 관리
- 수동으로 메모리를 저장하고 불러오는 과정

## 환경 설정

In [1]:
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv()

True

## 필요한 라이브러리 임포트 및 기본 설정

In [2]:
from operator import itemgetter
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델을 초기화합니다.
model = ChatOpenAI()

# 대화형 프롬프트를 생성합니다.
# 이 프롬프트는 시스템 메시지, 이전 대화 내역, 그리고 사용자 입력을 포함합니다.
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="chat_history"),  # 대화 기록을 위한 placeholder
        ("human", "{input}"),
    ]
)

## ConversationBufferMemory 생성

대화 내용을 저장할 메모리를 생성합니다.

- `return_messages=True`: 메시지 형태로 반환하도록 설정
- `memory_key="chat_history"`: 프롬프트의 MessagesPlaceholder와 일치하는 키 설정

In [3]:
# 대화 버퍼 메모리를 생성하고, 메시지 반환 기능을 활성화합니다.
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

C:\Users\cava2\AppData\Local\Temp\ipykernel_11084\2260825137.py:2: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")


In [4]:
# 현재 메모리 상태 확인 (비어있음)
memory.load_memory_variables({})

{'chat_history': []}

## RunnablePassthrough를 사용한 메모리 연결

`RunnablePassthrough.assign`을 사용하여 입력 데이터에 대화 기록을 추가합니다.

In [5]:
# RunnablePassthrough를 사용하여 chat_history 변수에 메모리 내용을 할당
runnable = RunnablePassthrough.assign(
    chat_history=RunnableLambda(memory.load_memory_variables)
    | itemgetter("chat_history")  # memory_key와 동일하게 입력
)

In [6]:
# runnable 테스트 - input과 함께 chat_history가 추가됨
runnable.invoke({"input": "hi"})

{'input': 'hi', 'chat_history': []}

## 전체 체인 구성

In [9]:
# runnable -> prompt -> model 순서로 체인 구성
chain = runnable | prompt | model

## 메모리를 활용한 대화 시작

In [10]:
# 첫 번째 대화: 자기소개
response = chain.invoke({"input": "만나서 반갑습니다. 제 이름은 홍기입니다."})
print(response.content)

만나서 반가워요, 홍기님! 무엇을 도와드릴까요?


In [11]:
# 아직 메모리에 저장하지 않았으므로 비어있음
memory.load_memory_variables({})

{'chat_history': []}

## 대화 내용을 메모리에 저장

In [12]:
# 입력과 응답을 메모리에 저장
memory.save_context(
    {"human": "만나서 반갑습니다. 제 이름은 홍기입니다."}, 
    {"ai": response.content}
)

# 저장된 대화 기록 확인
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='만나서 반갑습니다. 제 이름은 홍기입니다.', additional_kwargs={}, response_metadata={}),
  AIMessage(content='만나서 반가워요, 홍기님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={})]}

## 메모리를 활용한 연속 대화

In [13]:
# 이전 대화 내용을 기억하는지 확인
response = chain.invoke({"input": "제 이름이 무엇이었는지 기억하세요?"})
print(response.content)

네, 홍기님의 이름을 기억하고 있어요. 어떻게 도와드릴까요?


## 요약

이 노트북에서는 LCEL 체인에 메모리를 통합하는 방법을 학습했습니다:

1. **ConversationBufferMemory**: 대화 내용을 저장하는 메모리 클래스
2. **MessagesPlaceholder**: 프롬프트 템플릿에서 대화 기록을 삽입할 위치 지정
3. **RunnablePassthrough.assign**: 입력 데이터에 메모리 내용을 추가
4. **수동 메모리 관리**: `save_context()`를 통해 대화 내용을 명시적으로 저장

### 주의사항
- 현재 LCEL에서는 메모리를 수동으로 관리해야 합니다
- 각 대화 후 `save_context()`를 호출하여 대화 내용을 저장해야 합니다
- memory_key와 MessagesPlaceholder의 variable_name이 일치해야 합니다